In [1]:
using CSV, DelimitedFiles
using BenchmarkTools, Logging, Dates
using FreqTables
# For random variable creation 
using Distributions   
  
# For basic statistical operations 
using StatsBase 
  
# For reading and writing CSV files 
using CSV   
  
# For creation of Data Structures   
using DataFrames   
  
# For representing various plots 
using StatsPlots 

┌ Info: Precompiling FreqTables [da1fdf0e-e0ff-5433-a45f-9bb5ff651cb1]
└ @ Base loading.jl:1278
┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1278
┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1278


## Loading Datasets

In [56]:
dfForecastLoad = CSV.read("inputs/DEC_BA_PM(Forecast Load 2019).csv", DataFrame; dateformat = "mm/dd/yyyy HH:MM", types = Dict(:DT => Date));
dfBreakPtsSummer = CSV.read("inputs/HR_Breakpoints_Summer_DEC-DEP_(2019).csv", DataFrame; dateformat = "mm/dd/yyyy", types = Dict(:BEGIN_DATE => Date));
dfBreakPtsWinter = CSV.read("inputs/HR_Breakpoints_Winter_DEC-DEP_(2019).csv", DataFrame; dateformat = "mm/dd/yyyy", types = Dict(:BEGIN_DATE => Date));
dfUnitLookUpTb = CSV.read("inputs/UnitLookupAndDetailTable_(DEC-DEP).CSV", DataFrame; dateformat = "mm/dd/yyyy HH:MM:SS p", types = Dict(:BEGIN_DATE => Date, :END_DATE => Date ));
dfUnitSch = CSV.read("inputs/UnitSchedule_(DEC-DEP 2019).CSV", DataFrame; dateformat = "mm/dd/yyyy HH:MM:SS p", types = Dict(:BEGIN_DATE => Date));

dfHydroUnitRating = CSV.read("inputs/xHydroUnitRating_(DEC 2019).csv", DataFrame);
dfUnitCycleDEC = CSV.read("inputs/xUnitCycleLimit_(DEC 2019).csv", DataFrame);
dfUnitCycleDEP = CSV.read("inputs/xUnitCycleLimit_(DEP 2019).csv", DataFrame);
dfUnitRateSummer = CSV.read("inputs/xUnitRating_DEP-DEC-Summer (2019).csv", DataFrame);
dfUnitRateWinter = CSV.read("inputs/xUnitRating_DEP-DEC-Winter(2019).csv", DataFrame);

dfUnitStartUpDEC = CSV.read("inputs/xUnitStartupCharacteristic_(DEC 2019)_REV2.csv", DataFrame; dateformat = "mm/dd/yyyy HH:MM:SS p", types = Dict(:BEGIN_DATE => Date));

dfUnitStartUpDEP = CSV.read("inputs/xUnitStartupCharacteristic_(PEC 2019)_REV2.csv", DataFrame; dateformat = "mm/dd/yyyy HH:MM:SS p", types = Dict(:BEGIN_DATE => Date));

### File: DEC_BA_PM(Forecast Load 2019).csv

In [3]:
# number of rows and columns 
size(dfForecastLoad)

(8760, 35)

In [4]:
describe(dfForecastLoad)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,Region,,DEC,,DEC,1,,String
2,Model,,UC,,UC,1,,String
3,ExportId,,DEC_BA_PM,,DEC_BA_PM,1,,String
4,DT,,2019-01-01,,2019-12-31,365,,Date
5,Year,2019.0,2019,2019.0,2019,,,Int64
6,Month,6.52603,1,7.0,12,,,Int64
7,Day,15.7205,1,16.0,31,,,Int64
8,Hour,11.5,0,11.5,23,,,Int64
9,Load_Actual,12188.8,7395,11609.5,20597,,,Int64


In [5]:
first(dfForecastLoad, 5)

,Region,Model,ExportId,DT,Year,Month,Day,Hour,Load_Actual
,String,String,String,Date,Int64,Int64,Int64,Int64,Int64
1,DEC,UC,DEC_BA_PM,2019-01-01,2019,1,1,0,8377
2,DEC,UC,DEC_BA_PM,2019-01-01,2019,1,1,1,8035
3,DEC,UC,DEC_BA_PM,2019-01-01,2019,1,1,2,7746
4,DEC,UC,DEC_BA_PM,2019-01-01,2019,1,1,3,7590
5,DEC,UC,DEC_BA_PM,2019-01-01,2019,1,1,4,7549


### File: HR_Breakpoints_Summer_DEC-DEP_(2019).csv

In [6]:
size(dfBreakPtsSummer)

(1184, 14)

In [7]:
describe(dfBreakPtsSummer)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,119,,String
2,CC_KEY,0.472973,0,0.0,4,,,Int64
3,BEGIN_DATE,,2019-06-01,,2019-06-01,1,,Date
4,TIMEZONE,,EDT,,EDT,1,,String
5,EDITION,,Actual,,Actual,1,,String
6,NOLOAD_HEAT,447.538,7.7586,364.532,1598.7,,,Float64
7,NOLOAD_NOX,44.4789,0.0,22.008,464.916,,,Float64
8,NUM_POINTS,8.0,8,8.0,8,,,Int64
9,POINT,4.5,1,4.5,8,,,Int64


### File: HR_Breakpoints_Winter_DEC-DEP_(2019).csv

In [8]:
size(dfBreakPtsWinter)

(1192, 14)

In [9]:
describe(dfBreakPtsWinter)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,120,,String
2,CC_KEY,0.469799,0,0.0,4,,,Int64
3,BEGIN_DATE,,2019-12-01,,2019-12-01,1,,Date
4,TIMEZONE,,EDT,,EDT,1,,String
5,EDITION,,Actual,,Actual,1,,String
6,NOLOAD_HEAT,435.706,7.7586,364.532,1758.58,,,Float64
7,NOLOAD_NOX,44.1635,0.0,21.9329,538.696,,,Float64
8,NUM_POINTS,8.0,8,8.0,8,,,Int64
9,POINT,4.5,1,4.5,8,,,Int64


In [10]:
first(dfBreakPtsWinter, 5)

,UNIT_NAME,CC_KEY,BEGIN_DATE,TIMEZONE,EDITION,NOLOAD_HEAT,NOLOAD_NOX,NUM_POINTS
,String,Int64,Date,String,String,Float64,Float64,Int64
1,ALLE_UN01,0,2019-12-01,EDT,Actual,146.117,32.1457,8
2,ALLE_UN01,0,2019-12-01,EDT,Actual,146.117,32.1457,8
3,ALLE_UN01,0,2019-12-01,EDT,Actual,146.117,32.1457,8
4,ALLE_UN01,0,2019-12-01,EDT,Actual,146.117,32.1457,8
5,ALLE_UN01,0,2019-12-01,EDT,Actual,146.117,32.1457,8


### File: UnitLookupAndDetailTable_(DEC-DEP).CSV

In [11]:
size(dfUnitLookUpTb)

(173, 18)

In [12]:
describe(dfUnitLookUpTb)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_KEY,298.22,129,323.0,483,,
2,CC_KEY,0.294798,0,0.0,4,,
3,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,157,
4,UNIT_MED_NAME,,ALLEN_UN_1,,WSP IC,154,
5,UNIT_DESCRIPTION,,Allen 1,,Weatherspoon CT 1-4,155,
6,BEGIN_DATE,,2000-01-01,,2020-12-01,19,
7,END_DATE,,2020-02-29,,2100-01-01,4,
8,ACTIVE,,N,,Y,2,
9,UNIT_TYPE,,Cogen,,Steam Dual Boiler,10,


In [13]:
first(dfUnitLookUpTb, 5)

,UNIT_KEY,CC_KEY,UNIT_NAME,UNIT_MED_NAME,UNIT_DESCRIPTION
,Int64,Int64,String,String,String
1,390,1,SCCP_CC01,SCCP_CC01,Sutton CCP CC01 (Aggregate)
2,390,2,SCCP_CC01,SCCP_CC01,Sutton CCP CC01 (Aggregate)
3,385,1,FAYE_CC01,FAYE_CC01,PEC Butler Warner plant (Aggregate)
4,385,2,FAYE_CC01,FAYE_CC01,PEC Butler Warner plant (Aggregate)
5,385,3,FAYE_CC01,FAYE_CC01,PEC Butler Warner plant (Aggregate)


In [14]:
#CSV.write("List of UnitsFile Name.csv", df)

LoadError: UndefVarError: df not defined

### File: UnitSchedule_(DEC-DEP 2019).CSV

In [15]:
size(dfUnitSch)

(1600415, 14)

In [16]:
describe(dfUnitSch)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,PORTFOLIO_NAME,,DEC,,PEC,2,
2,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,136,
3,CC_KEY,0.310058,0,0.0,4,,
4,UNIT_TYPE,,Cogen,,Steam Dual Boiler,9,
5,BEGIN_DATE,,2019-01-01,,2019-12-31,365,
6,EDITION_NAME,,Actual,,Forecast,2,
7,MW,169.639,0,0.0,14409,,
8,BALDN,0.0,0,0.0,0,,
9,BALUP,0.0,0,0.0,0,,


In [17]:
first(dfUnitSch, 5)

,PORTFOLIO_NAME,UNIT_NAME,CC_KEY,UNIT_TYPE,BEGIN_DATE,EDITION_NAME,MW,BALDN
,String,String,Int64,String,Date,String,Int64,Int64
1,DEC,ALLE_UN01,0,Steam,2019-01-01,Actual,0,0
2,DEC,ALLE_UN01,0,Steam,2019-01-01,Actual,0,0
3,DEC,ALLE_UN01,0,Steam,2019-01-01,Actual,0,0
4,DEC,ALLE_UN01,0,Steam,2019-01-01,Actual,0,0
5,DEC,ALLE_UN01,0,Steam,2019-01-01,Actual,0,0


**Frequency table of units and number of records of actual and forecast schedules**

In [18]:
freqtable(dfUnitSch.UNIT_NAME, dfUnitSch.EDITION_NAME)

136×2 Named Array{Int64,2}
Dim1 ╲ Dim2 │   Actual  Forecast
────────────┼───────────────────
ALLE_UN01   │     8760         0
ALLE_UN02   │     8760         0
ALLE_UN03   │     8760         0
ALLE_UN04   │     8760         0
ALLE_UN05   │     8760         0
ASHV_CC01   │     3836      3379
ASHV_CC02   │     3836      3379
ASHV_CT03   │     8760         0
ASHV_CT04   │     8760         0
ASHV_UN01   │     8760         0
ASHV_UN02   │     8760         0
BELE_UN01   │     8760         0
⋮                    ⋮         ⋮
SPRT_CG01   │     8760      8759
SUTT_CT02   │       11         0
SUTT_CT04   │     8760         0
SUTT_CT05   │     8760         0
TILL_HYDR   │     8760      8759
WALT_HYDR   │     8760      8759
WCOP_CT10   │     8760         0
WCOP_CT11   │     8760         0
WCOP_CT12   │     8760         0
WCOP_CT13   │     8760         0
WCOP_CT14   │     8760         0
WSPN_CT01   │     8244         0

### File: xHydroUnitRating_(DEC 2019).csv

In [19]:
size(dfHydroUnitRating)

(8, 11)

In [20]:
describe(dfHydroUnitRating)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,UNIT_NAME,,BADC_PS01,,JOCA_PS04,8,,String
2,CC_KEY,0.0,0,0.0,0,,,Int64
3,EDITION,,Actual,,Actual,1,,String
4,TIMEZONE,,EDT,,EDT,1,,String
5,BEGIN_DATE,,4/1/2017 4:00,,4/1/2017 4:00,1,,String
6,PUMP_MIN,202.5,155,202.5,250,,,Int64
7,LOW_LFC,202.5,155,202.5,250,,,Int64
8,MAX_CAP,267.5,195,267.5,340,,,Int64
9,PUMP_MAX,267.5,195,267.5,340,,,Int64


### File: xUnitCycleLimit_(DEC 2019).csv

In [21]:
size(dfUnitCycleDEC)

(64, 8)

In [22]:
describe(dfUnitCycleDEC)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ALLE_UN01,,RCKN_CT05,61,
2,CC_KEY,0.140625,0,0.0,2,,
3,EDITION,,Actual,,Actual,1,
4,MIN_UPTIME,29.3906,1,2.0,120,,
5,MIN_DNTIME,8.35938,1,1.0,48,,
6,MAXRAMPRATE,8.72656,1.5,8.0,20.0,,
7,TIMEZONE,,EDT,,EDT,1,
8,BEGIN_DATE,,1/1/2011 5:00,,9/11/2015 4:00,12,


### File: xUnitCycleLimit_(DEP 2019).csv

In [23]:
size(dfUnitCycleDEP)

(79, 8)

In [24]:
describe(dfUnitCycleDEP)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ASHV_CT03,,WSPN_CT01,66,
2,CC_KEY,0.506329,0,0.0,4,,
3,EDITION,,Actual,,Actual,1,
4,MIN_UPTIME,19.1139,1,2.0,120,,
5,MIN_DNTIME,3.98734,1,2.0,12,,
6,MAXRAMPRATE,10.7063,0.3,10.0,20.0,,
7,TIMEZONE,,EDT,,EDT,1,
8,BEGIN_DATE,,10/18/2014 4:00,,9/25/2013 4:00,14,


### File: xUnitRating_DEP-DEC-Summer (2019)

In [25]:
size(dfUnitRateSummer)

(141, 11)

In [26]:
describe(dfUnitRateSummer)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,125,
2,CC_KEY,0.347518,0,0.0,4,,
3,EDITION,,Actual,,Actual,1,
4,TIMEZONE,,EDT,,EDT,1,
5,BEGIN_DATE,,04/01/2018 00,,04/01/2019 00,2,
6,EMERGENCY_MIN,114.298,0,74.0,650,,
7,NORMAL_MIN,114.44,0,74.0,650,,
8,NORMAL_MAX,261.66,4.0,159.0,1236.5,,
9,EMERGENCY_MAX,261.66,4.0,159.0,1236.5,,


### File: xUnitRating_DEP-DEC-Winter(2019).csv

In [27]:
size(dfUnitRateWinter)

(141, 11)

In [28]:
describe(dfUnitRateWinter)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ALLE_UN01,,WSPN_CT01,125,
2,CC_KEY,0.347518,0,0.0,4,,
3,EDITION,,Actual,,Actual,1,
4,TIMEZONE,,EDT,,EST,2,
5,BEGIN_DATE,,04/01/2018 00,,11/18/2018 00,2,
6,EMERGENCY_MIN,119.333,0,97.0,650,,
7,NORMAL_MIN,118.411,0,97.0,650,,
8,NORMAL_MAX,276.632,4.0,178.0,1236.5,,
9,EMERGENCY_MAX,276.632,4.0,178.0,1236.5,,


### File: xUnitStartupCharacteristic_(DEC 2019).csv

In [57]:
size(dfUnitStartUpDEC)

(161, 17)

In [58]:
describe(dfUnitStartUpDEC)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ALLE_UN01,,RCKN_CT05,65,
2,CC_KEY,0.111801,0,0.0,2,,
3,EDITION,,Actual,,Actual,1,
4,STARTUP_FUEL_NAME,,HYDR_KEOW,,SOLR_DEC_C,24,
5,MARKET_OR_CONTRACT,,Contract,,Contract,1,
6,HOUR_1,8.17391,8,8.0,12,,
7,HOUR_2,53.1677,32,32.0,76,,
8,HOUR_3,106.733,72,72.0,144,,
9,MBTU_1,1719.6,0,1037.0,8810,,


In [59]:
show(dfUnitStartUpDEC[1:5,:], allcols=true)

5×17 DataFrame
│ Row │ UNIT_NAME │ CC_KEY │ EDITION │ STARTUP_FUEL_NAME │ MARKET_OR_CONTRACT │
│     │ String    │ Int64  │ String  │ String            │ String             │
├─────┼───────────┼────────┼─────────┼───────────────────┼────────────────────┤
│ 1   │ ALLE_UN01 │ 0      │ Actual  │ LOIL_ALLE         │ Contract           │
│ 2   │ ALLE_UN01 │ 0      │ Actual  │ LOIL_ALLE         │ Contract           │
│ 3   │ ALLE_UN01 │ 0      │ Actual  │ LOIL_ALLE         │ Contract           │
│ 4   │ ALLE_UN01 │ 0      │ Actual  │ LOIL_ALLE         │ Contract           │
│ 5   │ ALLE_UN01 │ 0      │ Actual  │ LOIL_ALLE         │ Contract           │

│ Row │ HOUR_1 │ HOUR_2 │ HOUR_3 │ MBTU_1 │ MBTU_2 │ MBTU_3 │ COST_1 │ COST_2 │
│     │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │
├─────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│ 1   │ 8      │ 76     │ 144    │ 1046   │ 1594   │ 1822   │ 0      │ 0      │
│ 2   │ 8      │ 76     

### File: xUnitStartupCharacteristic_(DEP 2019).csv

In [60]:
size(dfUnitStartUpDEP)

(237, 17)

In [61]:
describe(dfUnitStartUpDEP)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Nothing
1,UNIT_NAME,,ASHV_CC01,,WSPN_CT01,72,
2,CC_KEY,0.518987,0,0.0,4,,
3,EDITION,,Actual,,Actual,1,
4,STARTUP_FUEL_NAME,,CCOA_MISC_CG01,,SOLR_PEC_C,36,
5,MARKET_OR_CONTRACT,,Contract,,Contract,1,
6,HOUR_1,8.16878,8,8.0,12,,
7,HOUR_2,33.8565,32,32.0,60,,
8,HOUR_3,86.1772,72,72.0,144,,
9,MBTU_1,685.439,0,244.0,3897,,


In [62]:
show(dfUnitStartUpDEP[1:5,:], allcols=true)

5×17 DataFrame
│ Row │ UNIT_NAME │ CC_KEY │ EDITION │ STARTUP_FUEL_NAME │ MARKET_OR_CONTRACT │
│     │ String    │ Int64  │ String  │ String            │ String             │
├─────┼───────────┼────────┼─────────┼───────────────────┼────────────────────┤
│ 1   │ ASHV_CC01 │ 1      │ Actual  │ NGAS_ASHV_CC01    │ Contract           │
│ 2   │ ASHV_CC01 │ 1      │ Actual  │ NGAS_ASHV_CC01    │ Contract           │
│ 3   │ ASHV_CC02 │ 1      │ Actual  │ NGAS_ASHV_CC02    │ Contract           │
│ 4   │ ASHV_CC02 │ 1      │ Actual  │ NGAS_ASHV_CC02    │ Contract           │
│ 5   │ ASHV_CT03 │ 0      │ Actual  │ NGAS_ASHV         │ Contract           │

│ Row │ HOUR_1 │ HOUR_2 │ HOUR_3 │ MBTU_1 │ MBTU_2 │ MBTU_3 │ COST_1  │
│     │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │ Int64  │ Float64 │
├─────┼────────┼────────┼────────┼────────┼────────┼────────┼─────────┤
│ 1   │ 8      │ 32     │ 72     │ 1249   │ 1249   │ 1249   │ 0.0     │
│ 2   │ 8      │ 32     │ 72     │ 1249   │ 1249   │ 124